In [17]:
import cProfile
from abc import ABC, abstractmethod

In [18]:
my_arr = [4, 5, 6, [4, 7], 7, 4, [2, 8], 7, [4, 7, [3, 8, 5]], 9, 7, 8,
          [4, 9, [3, 9, [4, 8, [3, 6, 7, 3], 3], 5], 4], 9]

In [19]:
class AbstractFlatten(ABC):
    """
    Abstract class for sorting
    """
    @classmethod
    @abstractmethod
    def flatten(arr: list):
        """Method to unpack/flatten the nested array"""
        pass

    @classmethod
    @abstractmethod
    def run(n: int = 50, print_output: bool = True):
        """Method to run flatten multiple times"""
        pass

    @classmethod
    @abstractmethod
    def test_flatten():
        """Method to test flatten()"""
        pass

    @classmethod
    def run_benchmark(cls, n: int = 50):
        for _ in range(n):
            cls.run(print_output=False)

In [20]:
class NonRecursiveFlatten(AbstractFlatten):
    @classmethod
    def flatten(cls, arr=my_arr):
        nested = True
        while nested:
            new = []
            nested = False
            for i in arr:
                if isinstance(i, list):
                    new.extend(i)
                    nested = True
                else:
                    new.append(i)
            arr = new
        return arr

    @classmethod
    def run(cls, n: int = 50, print_output: bool = True):
        for i in range(n):
            new_list = NonRecursiveFlatten.flatten(arr=my_arr)
        return new_list

    @classmethod
    def test_flatten(*args):
        assert NonRecursiveFlatten.flatten() == [4, 5, 6, 4, 7, 7, 4, 2, 8, 7, 4, 7, 3, 8,
                                                 5, 9, 7, 8, 4, 9, 3, 9, 4, 8, 3, 6, 7, 3, 3, 5, 4, 9]

In [21]:
NonRecursiveFlatten.test_flatten()

In [22]:
class RecursiveFlatten(AbstractFlatten):
    @classmethod
    def flatten(cls, arr=my_arr):
        new_list = []
        for i in arr:
            if isinstance(i, list):
                new_list.extend(RecursiveFlatten.flatten(i))
            else:
                new_list.append(i)
        return new_list

    @classmethod
    def run(cls, n: int = 50, print_output: bool = True):
        for i in range(n):
            new_list = RecursiveFlatten.flatten(arr=my_arr)
        return new_list

    @classmethod
    def test_flatten(*args):
        assert RecursiveFlatten.flatten() == [4, 5, 6, 4, 7, 7, 4, 2, 8, 7, 4, 7, 3, 8,
                                              5, 9, 7, 8, 4, 9, 3, 9, 4, 8, 3, 6, 7, 3, 3, 5, 4, 9]

In [23]:
RecursiveFlatten.test_flatten()

In [24]:
%timeit NonRecursiveFlatten.run(print_output=False)

478 µs ± 4.61 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [25]:
%timeit RecursiveFlatten.run(print_output=False)

224 µs ± 2.5 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [26]:
cProfile.run('NonRecursiveFlatten.run_benchmark()', sort='ncalls')

         612554 function calls in 0.119 seconds

   Ordered by: call count

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
   305000    0.024    0.000    0.024    0.000 {built-in method builtins.isinstance}
   285000    0.021    0.000    0.021    0.000 {method 'append' of 'list' objects}
    20000    0.002    0.000    0.002    0.000 {method 'extend' of 'list' objects}
     2500    0.071    0.000    0.118    0.000 1877689050.py:2(flatten)
       50    0.001    0.000    0.119    0.002 1877689050.py:17(run)
        1    0.000    0.000    0.119    0.119 803643361.py:23(run_benchmark)
        1    0.000    0.000    0.119    0.119 <string>:1(<module>)
        1    0.000    0.000    0.119    0.119 {built-in method builtins.exec}
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}




In [27]:
cProfile.run('RecursiveFlatten.run_benchmark()', sort='ncalls')

         222554 function calls (202554 primitive calls) in 0.049 seconds

   Ordered by: call count

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
   100000    0.008    0.000    0.008    0.000 {built-in method builtins.isinstance}
    80000    0.006    0.000    0.006    0.000 {method 'append' of 'list' objects}
22500/2500    0.032    0.000    0.048    0.000 1557871206.py:2(flatten)
    20000    0.002    0.000    0.002    0.000 {method 'extend' of 'list' objects}
       50    0.001    0.000    0.049    0.001 1557871206.py:12(run)
        1    0.000    0.000    0.049    0.049 803643361.py:23(run_benchmark)
        1    0.000    0.000    0.049    0.049 <string>:1(<module>)
        1    0.000    0.000    0.049    0.049 {built-in method builtins.exec}
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}




#### Здесь без lprun, который у меня не работает, тяжело определить, скорее всего рекурсивная функция работает быстрее как раз из-за многократного рекурсивного вызова, плюс в ней .append меньше раз вызывается, .isinstance тоже меньше раз вызывается. В целом, думаю, чем больше количество итераций с добавлением массива с глубиной 5 вложений, как я добавлял перед исправлениями в методе run(), тем больше лишних проходов нерекурсивной функции по этим вышеуказанным методам, что будет все сильнее замедлять ее по отношению к рекурсивной

In [28]:
%load_ext memory_profiler

The memory_profiler extension is already loaded. To reload it, use:
  %reload_ext memory_profiler


In [29]:
%%writefile run_nonrecursive_flatten.py

from memory_profiler import profile

my_arr = [4, 5, 6, [4, 7], 7, 4, [2, 8], 7, [4, 7, [3, 8, 5]], 9, 7, 8,
          [4, 9, [3, 9, [4, 8, [3, 6, 7, 3], 3], 5], 4], 9]


def nonrecursive_flatten(arr):
    nested = True
    while nested:
        new = []
        nested = False
        for i in arr:
            if isinstance(i, list):
                new.extend(i)
                nested = True
            else:
                new.append(i)
        arr = new
    return arr


@profile
def run_nonrecursive_flatten():
    for i in range(1000):
        result = nonrecursive_flatten(arr=my_arr)
    return result


if __name__ == '__main__':
    run_nonrecursive_flatten()

Overwriting run_nonrecursive_flatten.py


In [30]:
!python run_nonrecursive_flatten.py

Filename: C:\Users\Мария\PycharmProjects\aleksandr_smolin_python_hw\hw3_profiling\run_nonrecursive_flatten.py

Line #    Mem usage    Increment  Occurrences   Line Contents
    23     41.4 MiB     41.4 MiB           1   @profile
    24                                         def run_nonrecursive_flatten():
    25     41.4 MiB      0.0 MiB        1001       for i in range(1000):
    26     41.4 MiB      0.0 MiB        1000           result = nonrecursive_flatten(arr=my_arr)
    27     41.4 MiB      0.0 MiB           1       return result




In [31]:
%%writefile run_recursive_flatten.py

from memory_profiler import profile

my_arr = [4, 5, 6, [4, 7], 7, 4, [2, 8], 7, [4, 7, [3, 8, 5]], 9, 7, 8,
          [4, 9, [3, 9, [4, 8, [3, 6, 7, 3], 3], 5], 4], 9]


def recursive_flatten(arr):
    new_list = []
    for i in arr:
        if isinstance(i, list):
            new_list.extend(recursive_flatten(i))
        else:
            new_list.append(i)
    return new_list


@profile
def run_recursive_flatten():
    for i in range(1000):
        result = recursive_flatten(arr=my_arr)
    return result


if __name__ == '__main__':
    run_recursive_flatten()

Overwriting run_recursive_flatten.py


In [32]:
!python run_recursive_flatten.py

Filename: C:\Users\Мария\PycharmProjects\aleksandr_smolin_python_hw\hw3_profiling\run_recursive_flatten.py

Line #    Mem usage    Increment  Occurrences   Line Contents
    18     41.6 MiB     41.6 MiB           1   @profile
    19                                         def run_recursive_flatten():
    20     41.6 MiB      0.0 MiB        1001       for i in range(1000):
    21     41.6 MiB      0.0 MiB        1000           result = recursive_flatten(arr=my_arr)
    22     41.6 MiB      0.0 MiB           1       return result




#### Не прослеживаются изменения в потреблении памяти в большом диапазоне